Playing [Quordle](https://www.quordle.com/#/) or [Wordle](https://www.nytimes.com/games/wordle/index.html) and in a hurry? Want just one hint? Try this!

In [1]:
import numpy as np
import sys
import collections
from urllib.request import urlopen

print(sys.version, sys.executable)

3.8.9 (default, Oct 26 2021, 07:25:54) 
[Clang 13.0.0 (clang-1300.0.29.30)] /Users/swkar/wordle/env/bin/python


In [2]:
with urlopen('https://github.com/dwyl/english-words/raw/master/words.txt') as f:
    words_raw = f.read().decode('utf-8').split('\n')
print(type(words_raw), len(words_raw), words_raw[:10])

<class 'list'> 466551 ['2', '1080', '&c', '10-point', '10th', '11-point', '12-point', '16-point', '18-point', '1st']


In [3]:
words = list(set([w.upper()
               for w in words_raw
               if len(w)==5 and w.isalpha()
              ]))

# check wordlist
print(type(words), len(words), words[:10])

<class 'list'> 21952 ['UNBOW', 'GRAPH', 'MENSE', 'CASTS', 'INVOY', 'CRAFT', 'PANDA', 'CATES', 'WILED', 'HUSHO']


In [4]:
def get_indices(words):
    index = {}
    for map_id in ('has_letter', 'has_letter_in_pos', 'dont_have_letter'):
        index[map_id] = collections.defaultdict(set)

    for n, word in enumerate(words):
        for pos, letter in enumerate(word):
            index['has_letter'][letter].add(n)
            index['has_letter_in_pos'][(letter, pos)].add(n)

    master_set = set(list(range(len(words))))
    for key, set_ in index['has_letter'].items():
        index['dont_have_letter'][key] = list(master_set - set_)

    index_ = index
    index = {}
    for map_id in ('has_letter', 'has_letter_in_pos', 'dont_have_letter'):
        index[map_id] = collections.defaultdict(list)
        for key, set_ in index_[map_id].items():
            index[map_id][key] = list(set_)
            
    return index

index = get_indices(words)

In [5]:
# check indices
print([words[n] for n in index['has_letter']['G'][:5]])
print([words[n] for n in index['has_letter_in_pos'][('G', 4)][:5]])
print([words[n] for n in index['dont_have_letter']['G'][:5]])

['GALEA', 'GRAPH', 'MEGGS', 'FUGIO', 'RIGOL']
['ARENG', 'SWUNG', 'IRREG', 'INORG', 'CHUNG']
['UNBOW', 'MENSE', 'CASTS', 'INVOY', 'CRAFT']


In [6]:
def sum_to_one(prob):
    sum_prob = np.sum(prob)
    if sum_prob==0:
        return prob*0
    return prob / np.sum(prob)


def play_game(inputs_and_feedbacks, verbose=0, n_hints=12):
    n_words = len(words)
    n_wordle = len(inputs_and_feedbacks[0][1])
    probs_ = np.ones((n_wordle, n_words))
    
    def get_num_choices():
        return np.prod([(np.count_nonzero(p)) for p in probs_])

    if verbose>=1:
        print(f'At start, num_choices per slot {n_words:,}, total {get_num_choices():,}')
    
    def print_hints():
        for n, p in enumerate(probs_):
            try:
                guess = set(np.random.choice(
                    words, size=n_hints, p=sum_to_one(p), replace=True))
            except:
                guess = set()
            guess_str = ', '.join(guess)
            print(f'{n}: {guess_str} ({len(guess)}/{np.count_nonzero(p):,})')
                
    for m, (word, feedbacks) in enumerate(inputs_and_feedbacks):
        word = word.upper()
        for n, feedback in enumerate(feedbacks):
            letters_present = set([letter 
                                   for letter, feedback_ in zip(word, feedback)
                                   if feedback_ != '-'])
            for pos, (letter, feedback_) in enumerate(zip(word, feedback)):
                if feedback_ == '-':
                    if letter not in letters_present:
                        not_possible = index['has_letter'][letter]
                        probs_[n, not_possible] = 0
                elif feedback_ == '+':
                    only_possible = index['has_letter_in_pos'][(letter, pos)]
                    new_probs_ = np.zeros_like(probs_[n])
                    new_probs_[only_possible] = probs_[n, only_possible]
                    probs_[n, :] = new_probs_
                elif feedback_ == '.':
                    not_possible = index['has_letter_in_pos'][(letter, pos)]
                    probs_[n, not_possible] = 0
                    not_possible = index['dont_have_letter'][letter]
                    probs_[n, not_possible] = 0

                if verbose>=2:
                    print(f'step {(m, n, pos, letter, feedback_)}, choices {np.count_nonzero(probs_[n]):,}')
        if verbose>=1:
            print(f'\nAfter feedback #{m}, num_choices are {get_num_choices():,}, some hints..')
            print_hints()
    if not verbose:
        print(f'\nAfter feedback #{m}, num_choices are {get_num_choices():,}, some hints..')
        print_hints()

In [7]:
# Put your Quordle feedback
play_game([('apple', ('.----', '----.', '---.+', '---+-')),
           ('birch', ('----.', '--.-.', '-----', '----.')),
           ('stout', ('..---', '+----', '+-.--', '--.--')),
           ('solve', ('.----', '+---.', '+++++', '-++--')),
           ('sheer', ('..---', '+++++', '',      '-.---')),
           ('holly', ('+---+', '',      '',      '+++++')),
          ], verbose=1)

At start, num_choices per slot 21,952, total 232,218,265,089,212,416

After feedback #0, num_choices are 895,229,301,600, some hints..
0: KAKIS, BOYAU, STATS, GABAR, CASON, NUBIA, STAFF, DIMNA, MANGI, FAFFY, CONDA, TRIGA (12/3,928)
1: BERCY, EVROS, EXCUD, TIECK, MUTES, OKETO, NERON, TETON, GIVEN, WESTY, CZECH, RIDES (12/2,825)
2: LUNGE, LONGE, BELUE, ELEME, LUCIE, LECCE, ROLFE, LOUIE, BLDGE, CLIVE (10/243)
3: ROILY, WRYLY, SWOLN, CULLY, SKULL, DOILY, WILLI, SLYLY, NOLLY, STOLL, WHILK, COOLS (12/332)

After feedback #1, num_choices are 17,889,032, some hints..
0: HAKAM, HANYA, HAKKA, HANNY, OHAUS, SHAHS, HAFTS, HAMMY, GOTHA, SHADS, WHANK, WHATA (12/238)
1: RHEEN, SHERD, KHMER, HUGER, HENRY, SHERM, TOHER, THEOR, ETHER, OSHER (10/43)
2: LOSSE, OKLEE, FLUKE, NOLTE, GOLEE, GLOME, LETTE, KLEVE, NELSE, LEGGE (10/92)
3: HOLLO, HOOLY, KOHLS, WHOLL, UHLLO, HOWLS, HOULT, SHULN, HOTLY (9/19)

After feedback #2, num_choices are 204, some hints..
0: GHAST, HASNT, HASTY, YASHT, THATS, THAWS, GHATS, H

In [8]:
# Also works for your Wordle feedback
play_game([('apple', ('.----',)),
           ('birch', ('----.',)),
           ('stout', ('..---',)),
           ('solve', ('.----',)),
           ('sheer', ('..---',)),
           ('holly', ('+---+',)),
          ], verbose=1)

At start, num_choices per slot 21,952, total 21,952

After feedback #0, num_choices are 3,928, some hints..
0: KUSAM, CANNY, DANNY, SAIDI, BAFFY, SHAUM, UMAUA, WRANS, BACIN, SANDS, QUASH, KOKRA (12/3,928)

After feedback #1, num_choices are 238, some hints..
0: OATHS, MAHAU, MOHAM, KOHUA, HAYDN, HASTY, HAGGY, HADJS, KHVAT, HUMAN, JOHAN, HANNO (12/238)

After feedback #2, num_choices are 17, some hints..
0: TASHA, MATHS, HASHT, HANTS, THATS, GHATS, KHATS, HAFTS (8/17)

After feedback #3, num_choices are 17, some hints..
0: HASNT, YASHT, HASHT, WHATS, GHATS, KHATS, HAFTS, HADST (8/17)

After feedback #4, num_choices are 11, some hints..
0: HASNT, HASTY, YASHT, HASHT, HANTS, HAFTS, HADST, HASTA (8/11)

After feedback #5, num_choices are 1, some hints..
0: HASTY (1/1)
